In [1]:
# Importando librerías
import urllib.request
import os
import pandas as pd
import zipfile
import datetime
import json

## Descargando datos

In [2]:
# Definiendo url, nombres de los archivos y dirección donde se descargarán
endireh_2021_url = "https://www.inegi.org.mx/contenidos/programas/endireh/2021/datosabiertos/conjunto_de_datos_endireh_2021_csv.zip"
endireh_2021_archivo = "endireh_2021.zip"
sesnsp_2021_url = "https://api.datamexico.org/tesseract/cubes/sesnsp_crimes/aggregate.jsonrecords?captions%5B%5D=Geography.Geography.State.State+slug+ES&captions%5B%5D=Geography.Geography.Municipality.Municipality+ES&captions%5B%5D=Type.Type.Crime+Type.Crime+Type+ES&cuts%5B%5D=Date.Date.Year.2021&drilldowns%5B%5D=Date.Date.Year&drilldowns%5B%5D=Geography.Geography.State&drilldowns%5B%5D=Geography.Geography.Municipality&drilldowns%5B%5D=Type.Type.Crime+Type&measures%5B%5D=Value&parents=false"
sesnsp_2021_archivo = "sesnsp_2021.json"
subdir = "./data/"

In [4]:
# Descargando datos y creando archivo .txt con la documentación correspondiente
if not os.path.exists(subdir + endireh_2021_archivo):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    urllib.request.urlretrieve(endireh_2021_url, subdir + endireh_2021_archivo)  
    with zipfile.ZipFile(subdir + endireh_2021_archivo, "r") as zip_ref:
        zip_ref.extractall(subdir)
    
    urllib.request.urlretrieve(sesnsp_2021_url, subdir + sesnsp_2021_archivo)  

    with open(subdir + "info.txt", 'w') as f:
        f.write("Archivos sobre incidencia delictiva y resultados de encuesta sobre violencia contra la mujer\n")
        info = """
        Se decargaron datos de dos fuentes de información distintas. Por un lado, los datos de incidencias 
        delictivas a nivel estatal, agrupados por tipos de delitos, por estados y por municipios durante 
        el año 2021, como fuente del Secretariado Ejecutivo del Sistema Nacional de Seguridad Pública (SESNSP).
        Estos datos se obtuvieron del sitio web https://datamexico.org
        
        Por otro lado, se encuentran los datos que recogen los resultados de la Encuesta Nacional sobre 
        la Dinámica de las Relaciones en los Hogares (ENDIREH) 2021; información sobre las experiencias 
        de violencia física, económica, sexual, psicológica y patrimonial que han enfrentado las mujeres 
        de 15 años y más en los ámbitos familiar, escolar, laboral, comunitario y de pareja. 
        Estos resultados fueron publicados en el sitio web https://www.inegi.org.mx
        """ 
        f.write(info + '\n')
        f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write("Desde: " + sesnsp_2021_url + "\n")
        f.write("Nombre: " + sesnsp_2021_archivo + "\n")
        f.write("Encuesta ENDIREH 2021: " + endireh_2021_url + "\n")
        f.write("Nombre: " + endireh_2021_archivo + "\n")

## Leyendo archivos descargados

#### ENDIREH 2021

In [5]:
# Obteniendo datos donde se resume la información de violencia de la encuesta endireh 2021
dir_VD = subdir + 'conjunto_de_datos_TB_VD/conjunto_de_datos/conjunto_de_datos_TB_VD.csv'
df_VD = pd.read_csv(dir_VD, encoding = 'latin_1')

df_VD.head()

,ID_VIV,ID_PER,UPM,VIV_SEL,HOGAR,N_REN,DOMINIO,CVE_ENT,NOM_ENT,CVE_MUN,...,VCOM_A,VCOM_12M,VFAM,VPAR_A,VPAR_12M,FAC_VIV,FAC_MUJ,ESTRATO,UPM_DIS,EST_DIS
0,100003.01,0100003.01.1.02\r,100003,1,1,2,U\r,1,AGUASCALIENTES\r,1,...,1,2,2,1\r,1\r,113,113,4,1,3
1,100003.02,0100003.02.1.02\r,100003,2,1,2,U\r,1,AGUASCALIENTES\r,1,...,2,2,2,1\r,2\r,113,113,4,1,3
2,100003.03,0100003.03.1.03\r,100003,3,1,3,U\r,1,AGUASCALIENTES\r,1,...,2,2,2,2\r,2\r,113,227,4,1,3
3,100003.05,0100003.05.1.02\r,100003,5,1,2,U\r,1,AGUASCALIENTES\r,1,...,1,1,1,2\r,2\r,113,113,4,1,3
4,100004.01,0100004.01.1.02\r,100004,1,1,2,U\r,1,AGUASCALIENTES\r,1,...,1,2,2,1\r,2\r,78,155,2,2,1


#### SESNSP 2021

In [6]:
# Obteniendo datos con la información de delitos ocurridos durante 2021
with open(subdir + sesnsp_2021_archivo, encoding = 'utf8') as fp:
    data_sesnsp = json.load(fp)

print("\nNúmero de entradas: {}".format(len(data_sesnsp)))
print("\nNombre de los atributos: {}".format(", ".join(data_sesnsp.keys())))


Número de entradas: 2

Nombre de los atributos: data, source


In [7]:
# Convirtiendo los datos en dataframe
df_sesnsp = pd.DataFrame(data_sesnsp['data'])
df_sesnsp.head()

,Year,State ID,State,Municipality ID,Municipality,Crime Type ID,Crime Type,Value
0,2021,1,aguascalientes-ag,1001,Aguascalientes,101,Abuso de Confianza,549
1,2021,1,aguascalientes-ag,1001,Aguascalientes,102,Daño a la Propiedad,3100
2,2021,1,aguascalientes-ag,1001,Aguascalientes,103,Despojo,279
3,2021,1,aguascalientes-ag,1001,Aguascalientes,104,Extorsión,53
4,2021,1,aguascalientes-ag,1001,Aguascalientes,105,Fraude,1919


## Transformando datos

#### ENDIREH 2021 

In [8]:
# Sustituyendo datos faltantes NaN por 9
df_VD = df_VD.fillna(9)

# Sustituyendo datos para cambiar su tipo
df_VD = df_VD.replace(['\r', '9\r', '1\r', '2\r'],[9, 9, 1, 2])
df_VD = df_VD.replace('\r', '', regex=True)

## Extrayendo información de los datos

#### SESNSP 2021

In [9]:
# Seleccionando crímenes de violencia contra la mujer
df_sesnsp_crimenes = df_sesnsp[df_sesnsp['Crime Type ID'] == 202]

df_sesnsp_crimenes.head()

,Year,State ID,State,Municipality ID,Municipality,Crime Type ID,Crime Type,Value
8,2021,1,aguascalientes-ag,1001,Aguascalientes,202,Violencia de Género en Todas sus Modalidades D...,3
48,2021,1,aguascalientes-ag,1002,Asientos,202,Violencia de Género en Todas sus Modalidades D...,0
88,2021,1,aguascalientes-ag,1003,Calvillo,202,Violencia de Género en Todas sus Modalidades D...,0
128,2021,1,aguascalientes-ag,1004,Cosío,202,Violencia de Género en Todas sus Modalidades D...,0
168,2021,1,aguascalientes-ag,1005,Jesús María,202,Violencia de Género en Todas sus Modalidades D...,0


In [10]:
# Cantidad de crímenes de violencia contra la mujer por estados
df_sesnsp_crimenes = df_sesnsp_crimenes.groupby(['State ID', 'State']).sum()[['Value']].reset_index()
df_sesnsp_crimenes.columns = ['CVE_ENT','ESTADO', 'DELITOS'] 
df_sesnsp_crimenes.head()

,CVE_ENT,ESTADO,DELITOS
0,1,aguascalientes-ag,4
1,2,baja-california-bc,0
2,3,baja-california-sur-bs,7
3,4,campeche-cm,0
4,5,coahuila-de-zaragoza-co,16


#### ENDIREH 2021

In [11]:
# Cantidad de mujeres encuestadas por estados
df_encuesta = df_VD.groupby(['CVE_ENT','NOM_ENT']).count()
df_encuesta = df_encuesta[['ID_PER']]
df_encuesta.head()

,,ID_PER
CVE_ENT,NOM_ENT,
1,AGUASCALIENTES,3536
2,BAJA CALIFORNIA,3226
3,BAJA CALIFORNIA SUR,3111
4,CAMPECHE,3557
5,COAHUILA DE ZARAGOZA,3462


In [12]:
# Cantidad de mujeres encuestadas que experimentaron violencia en los últimos 12 meses (2021)
violencia = df_VD[df_VD['VTOT_12M'] == 1]

In [13]:
# Cantidad de mujeres encuestadas que experimentaron violencia en los últimos 12 meses (2021)
# por estados
violencia_estados = violencia.groupby(['CVE_ENT','NOM_ENT']).count()
violencia_estados = violencia_estados[['ID_PER']]
violencia_estados.head()

,,ID_PER
CVE_ENT,NOM_ENT,
1,AGUASCALIENTES,1663
2,BAJA CALIFORNIA,1147
3,BAJA CALIFORNIA SUR,1169
4,CAMPECHE,1360
5,COAHUILA DE ZARAGOZA,1506


In [14]:
# Creando dataframe de prevalencia de violencia  en 2021 según la encuesta
violencia_prevalencia = pd.merge(violencia_estados, df_encuesta, on=['CVE_ENT','NOM_ENT'] )
violencia_prevalencia.columns = ['MUJERES_VIOLENTADAS', 'TOTAL_MUJERES']     

# Tasa de prevalencia de la violencia: MUJERES_VIOLENTADAS/TOTAL_MUJERES
violencia_prevalencia['PREVALENCIA_VIOLENCIA'] =  violencia_prevalencia['MUJERES_VIOLENTADAS']/violencia_prevalencia['TOTAL_MUJERES']
 
# Eliminando columnas como indices    
violencia_prevalencia = violencia_prevalencia.reset_index()

In [15]:
# Creando dataframe final sobre la relacion de crímenes de violencia contra la mujer y la prevalencia
# de violencia según encuesta
df = pd.merge(violencia_prevalencia, df_sesnsp_crimenes, on=['CVE_ENT'])
df = df[['CVE_ENT','NOM_ENT', 'MUJERES_VIOLENTADAS', 'TOTAL_MUJERES', 'PREVALENCIA_VIOLENCIA', 'DELITOS']]

df.head()

,CVE_ENT,NOM_ENT,MUJERES_VIOLENTADAS,TOTAL_MUJERES,PREVALENCIA_VIOLENCIA,DELITOS
0,1,AGUASCALIENTES,1663,3536,0.470305,4
1,2,BAJA CALIFORNIA,1147,3226,0.355549,0
2,3,BAJA CALIFORNIA SUR,1169,3111,0.375763,7
3,4,CAMPECHE,1360,3557,0.382345,0
4,5,COAHUILA DE ZARAGOZA,1506,3462,0.435009,16


## Diccionario de datos

In [16]:
# Creando dataframe con diccionario de datos
dic = pd.DataFrame({
    'Tipo': df.dtypes,
    'Descripcion': ['Codigo del estado', 'Nombre del estado', 
                   'Cantidad de mujeres encuestadas víctimas de violencia',
                   'Total de mujeres encuestadas', 
                   'Tasa entre la cantidad de mujeres violentadas y el total de encuestadas',
                   'Cantidad de delitos reportados de violencia de género'
                   ],
    'Fuente': ['ENDIREH 2021: https://www.inegi.org.mx,  SESNSP 2021: https://datamexico.org',
               'ENDIREH 2021: https://www.inegi.org.mx', 'ENDIREH 2021: conjunto_de_datos_TB_VD',
               'ENDIREH 2021: conjunto_de_datos_TB_VD', 'Tasa entre los datos de mujeres violentadas y encuestadas de ENDIREH 2021: conjunto_de_datos_TB_VD',
               'SESNSP 2021: Tipo de crimen (Violencia de Género en Todas sus Modalidades Distintas a la Violencia Familiar)'
              ]
})

In [17]:
# Visualizando diccionario
dic

,Tipo,Descripcion,Fuente
CVE_ENT,int64,Codigo del estado,"ENDIREH 2021: https://www.inegi.org.mx, SESNS..."
NOM_ENT,object,Nombre del estado,ENDIREH 2021: https://www.inegi.org.mx
MUJERES_VIOLENTADAS,int64,Cantidad de mujeres encuestadas víctimas de vi...,ENDIREH 2021: conjunto_de_datos_TB_VD
TOTAL_MUJERES,int64,Total de mujeres encuestadas,ENDIREH 2021: conjunto_de_datos_TB_VD
PREVALENCIA_VIOLENCIA,float64,Tasa entre la cantidad de mujeres violentadas ...,Tasa entre los datos de mujeres violentadas y ...
DELITOS,int64,Cantidad de delitos reportados de violencia de...,SESNSP 2021: Tipo de crimen (Violencia de Géne...
